In [2]:
import os
import math
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import math

In [3]:
from sklearn.discriminant_analysis import StandardScaler


class CustomDataset(Dataset):
    def __init__(self, csv_path="data/dataset_train_2024.csv"):
        # Load data from CSV
        data = pd.read_csv(csv_path)
        
        # Extract features
        self.sequences_1 = data.iloc[:, 1:129].values * 100  # Columns 1-128 (1-based indexing)
        self.sequences_2 = data.iloc[:, 129:257].values * 100  # Columns 129-256
        self.extra_feature = data.iloc[:, 257].values.reshape(-1, 1)  # Column 257

        # Combine features
        all_features = np.hstack([self.sequences_1, self.sequences_2, self.extra_feature])
        
        # Normalize features
        self.scaler = StandardScaler()
        self.normalized_features = self.scaler.fit_transform(all_features)
        self.features = torch.tensor(self.normalized_features, dtype=torch.float32)


        # Encode labels
        self.label_encoder = LabelEncoder()
        self.labels = torch.tensor(self.label_encoder.fit_transform(data.iloc[:, -1]), dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
    def inverseTransform(self, array):
        return self.label_encoder.inverse_transform(array)

In [4]:
class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional=False, dropout=0.0):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.rnn = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout
        )
        # Adjust the output dimension if bidirectional is used
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, x):
        # RNN forward pass
        # Input shape: [batch_size, seq_len, input_dim]
        out, _ = self.rnn(x)  # out: [batch_size, seq_len, hidden_dim]
        # Use the last hidden state for classification
        out = out[:, -1, :]  # [batch_size, hidden_dim]
        out = self.fc(out)  # [batch_size, num_classes]
        return out


In [5]:
def reformat_tensor(tensor):
    """
    Reformats a tensor from shape [batch_size, 257] to [batch_size, 128, 3] for RNN input.
    """
    batch_size = tensor.shape[0]
    # Extract sequences
    seq1 = tensor[:, :128]  # First 128 features
    seq2 = tensor[:, 128:256]  # Next 128 features
    # Extract noise and expand it to match seq_len
    noise = tensor[:, -1].unsqueeze(1).expand(batch_size, 128)
    # Stack along the last dimension to form [batch_size, 128, 3]
    return torch.stack([seq1, seq2, noise], dim=2)

# Hyperparameters
First try:

hidden_dim = 64

num_layers = 2

bidirectional = True  # Set to False for unidirectional RNN

num_classes = 5

learning_rate = 0.001

epochs = 50

batch_size = 32

Second try (and best):

hidden_dim = 32

num_layers = 3

bidirectional = True  # Set to False for unidirectional RNN

num_classes = 5

learning_rate = 0.001

epochs = 50

batch_size = 16

In [11]:
# Hyperparameters

hidden_dim = 256
num_layers = 3
bidirectional = True  # Set to False for unidirectional RNN
num_classes = 5
learning_rate = 0.001
epochs = 36
batch_size = 32

# Prepare Data
dataset = CustomDataset()
train_size = int(len(dataset))  # 80% for training
test_size = 0 # len(dataset) - train_size  # 20% for testing
train_data, test_data = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Model Initialization
model = RNNClassifier(
    input_dim=3,  # This matches the last dimension of [batch_size, 128, 3]
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    bidirectional=bidirectional,
    dropout=0.3
).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
print("Training RNN model...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        features = reformat_tensor(features).to(device)  # Reshape for RNN
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for features, labels in test_loader:
        features = reformat_tensor(features).to(device)
        labels = labels.to(device)

        outputs = model(features)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")


Using device: cuda
Training RNN model...
Epoch 1/36, Loss: 1.3156
Epoch 2/36, Loss: 0.3862
Epoch 3/36, Loss: 0.2689
Epoch 4/36, Loss: 0.1483
Epoch 5/36, Loss: 0.1139
Epoch 6/36, Loss: 0.0706
Epoch 7/36, Loss: 0.0603
Epoch 8/36, Loss: 0.0432
Epoch 9/36, Loss: 0.0462
Epoch 10/36, Loss: 0.0287
Epoch 11/36, Loss: 0.0352
Epoch 12/36, Loss: 0.0347
Epoch 13/36, Loss: 0.0284
Epoch 14/36, Loss: 0.0283
Epoch 15/36, Loss: 0.0383
Epoch 16/36, Loss: 0.0332
Epoch 17/36, Loss: 0.0170
Epoch 18/36, Loss: 0.0230
Epoch 19/36, Loss: 0.0178
Epoch 20/36, Loss: 0.0254
Epoch 21/36, Loss: 0.0309
Epoch 22/36, Loss: 0.0129
Epoch 23/36, Loss: 0.0091
Epoch 24/36, Loss: 0.0167
Epoch 25/36, Loss: 0.0221
Epoch 26/36, Loss: 0.0190
Epoch 27/36, Loss: 0.0165
Epoch 28/36, Loss: 0.0202
Epoch 29/36, Loss: 0.0090
Epoch 30/36, Loss: 0.0133
Epoch 31/36, Loss: 0.0109
Epoch 32/36, Loss: 0.0342
Epoch 33/36, Loss: 0.0339
Epoch 34/36, Loss: 0.0168
Epoch 35/36, Loss: 0.0091
Epoch 36/36, Loss: 0.0074
Test Accuracy: nan


c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\numpy\lib\_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [13]:
from sklearn.metrics import f1_score
# Testing Loop
print("Testing the model...")
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for features, labels in test_loader:
        # Move data to the appropriate device
        features, labels = features.to(device), labels.to(device)
        
        # Reformat the input tensor to match the expected RNN input shape
        features = reformat_tensor(features)
        
        # Get model predictions
        outputs = model(features)
        _, preds = torch.max(outputs, dim=1)
        
        # Collect predictions and labels for evaluation
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Evaluate performance
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')  # or 'macro', 'micro', depending on your use case
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Testing the model...
F1 Score: nan
Accuracy: nan


c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\numpy\lib\_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Ricard\Documents\Git\MLEARN\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Prepare for Kaggle


In [14]:
#Using the model for prediction with the evaluation dataset

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Define the dataset class
class UnlabeledDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe

        self.scaler = StandardScaler()
        normalized_values = self.scaler.fit_transform(self.data.values)
        self.normalized_data = pd.DataFrame(
            normalized_values, columns=self.data.columns, index=self.data.index
        )
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.normalized_data.iloc[idx].values.astype('float32')  # Retrieve row as NumPy array
        if self.transform:
            inputs = self.transform(inputs)
        return inputs

# Load the unlabeled dataset
csv_path = "data/dataset_test_no_label_2024.csv"  # Path to the dataset CSV file
unlabeled_df = pd.read_csv(csv_path)  # Update the filename
unlabeled_df = unlabeled_df.drop(unlabeled_df.columns[0], axis=1)
unlabeled_dataset = UnlabeledDataset(unlabeled_df)
unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Store predictions and indices
predictions = []
indices = []

# Generate predictions
with torch.no_grad():
    for idx, inputs in enumerate(unlabeled_dataloader):
        inputs = inputs.to(device)  # Send inputs to the same device as the model
        
        # Reformat inputs to match RNN input shape
        inputs = reformat_tensor(inputs)
        
        # Forward pass
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)  # Get predicted class
        
        # Save predictions and indices
        start_idx = idx * unlabeled_dataloader.batch_size
        batch_indices = list(range(start_idx, start_idx + len(inputs)))  # Adjusting the index properly
        indices.extend(batch_indices)
        predictions.extend(preds.cpu().numpy())

# Create a DataFrame with indices and predictions
output_df = pd.DataFrame({"ID": indices, "MODULATION": dataset.inverseTransform(predictions)})

# Save to a CSV file
output_df.to_csv("predictions_RNN_4.csv", index=False)

print("Predictions saved to 'predictions_RNN.csv'")

Predictions saved to 'predictions_RNN.csv'


# Save the model

In [30]:
# Save the model's state dictionary
model_path = "./RNN/rnn_classifier_gridSearch_3_best.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to ./RNN/rnn_classifier_gridSearch_3_best.pth


To load the saved model we use 

In [12]:
# Define parameters
input_dim = 3  # Match your input dimensions
hidden_dim = 128  # Hidden dimension of your RNN
num_layers = 2  # Number of RNN layers
num_classes = 5  # Number of output classes
bidirectional = False  # Match your model setting

# Recreate the model
loaded_model = RNNClassifier(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes,
    bidirectional=bidirectional
)

# Load the state dictionary
loaded_model.load_state_dict(torch.load("rnn_classifier.pth"))
loaded_model.to(device)  # Send the model to the appropriate device
loaded_model.eval()  # Set the model to evaluation mode

print("Model loaded successfully!")

C:\Users\Ricard\AppData\Local\Temp\ipykernel_22920\4031045064.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("rnn_classifier.pt

FileNotFoundError: [Errno 2] No such file or directory: 'rnn_classifier.pth'

# GridSearch


In [22]:
def train_model(model, train_loader, val_loader, epochs, learning_rate, batch_size):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            features = reformat_tensor(features)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")
            

    # Evaluate the model
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            features = reformat_tensor(features)
            outputs = model(features)
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    score = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, score

In [35]:
from itertools import product

hidden_dims = [256, 512]
num_layers = [2, 3, 4, 5, 6]
learning_rates = [0.001]
batch_sizes = [32]
dropouts = [0.2, 0.5]
bidirectionals = [True]
num_classes = 5
epochs = 35
    
best_accuracy = 0
best_f1_score = 0
best_params = None

# Prepare Data
dataset = CustomDataset()
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing
train_data, test_data = random_split(dataset, [train_size, test_size])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Generate all combinations of hyperparameters
hyperparameter_combinations = product(hidden_dims, num_layers, learning_rates, batch_sizes, dropouts, bidirectionals)

for hidden_dim, num_layer, learning_rate, batch_size, dropout, bidirectional in hyperparameter_combinations:
        print(f"Training with hidden_dim={hidden_dim}, num_layers={num_layer}, lr={learning_rate}, batch_size={batch_size}, dropout={dropout}, bidirectional={bidirectional}")
        
        # Create DataLoader for current batch_size
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

        # Initialize and train the model
        model = RNNClassifier(input_dim=3, hidden_dim=hidden_dim, num_layers=num_layer, num_classes=num_classes, 
                              bidirectional=bidirectional, dropout=dropout).to(device)
        
        accuracy, score = train_model(model, train_loader, val_loader, epochs, learning_rate, batch_size)
        
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation F1-Score: {score:.4f}")
        
        # Update best accuracy and parameters
        if score > best_f1_score:
            best_accuracy = accuracy
            best_f1_score = score
            best_params = {
                  'hidden_dim': hidden_dim,
                  'num_layers': num_layer,
                  'learning_rate': learning_rate,
                  'batch_size': batch_size,
                  'dropout': dropout,
                  'bidirectional': bidirectional
                  }

print("Best Parameters found:", best_params)
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Best Validation F1-Score: {best_f1_score:.4f}")

Using device: cuda
Training with hidden_dim=256, num_layers=2, lr=0.001, batch_size=32, dropout=0.2, bidirectional=True
Epoch 1/35, Loss: 0.0271
Epoch 2/35, Loss: 0.0271
Epoch 3/35, Loss: 0.0271
Epoch 4/35, Loss: 0.0271
Epoch 5/35, Loss: 0.0271
Epoch 6/35, Loss: 0.0271
Epoch 7/35, Loss: 0.0271
Epoch 8/35, Loss: 0.0271
Epoch 9/35, Loss: 0.0271
Epoch 10/35, Loss: 0.0271
Epoch 11/35, Loss: 0.0271
Epoch 12/35, Loss: 0.0271
Epoch 13/35, Loss: 0.0271
Epoch 14/35, Loss: 0.0271
Epoch 15/35, Loss: 0.0271
Epoch 16/35, Loss: 0.0271
Epoch 17/35, Loss: 0.0271
Epoch 18/35, Loss: 0.0271
Epoch 19/35, Loss: 0.0271
Epoch 20/35, Loss: 0.0271
Epoch 21/35, Loss: 0.0271
Epoch 22/35, Loss: 0.0271
Epoch 23/35, Loss: 0.0271
Epoch 24/35, Loss: 0.0271
Epoch 25/35, Loss: 0.0271
Epoch 26/35, Loss: 0.0271
Epoch 27/35, Loss: 0.0271
Epoch 28/35, Loss: 0.0271
Epoch 29/35, Loss: 0.0271
Epoch 30/35, Loss: 0.0271
Epoch 31/35, Loss: 0.0271
Epoch 32/35, Loss: 0.0271
Epoch 33/35, Loss: 0.0271
Epoch 34/35, Loss: 0.0271
Epoch

: 

In [ ]:
print("Best Parameters found:", best_params)
print(f"Best Validation Accuracy: {best_accuracy:.4f}")
print(f"Best Validation F1-Score: {best_f1_score:.4f}")

RNN_GridSearch

Best Parameters found: {'hidden_dim': 32, 'num_layers': 3, 'learning_rate': 0.001, 'batch_size': 16, 
'dropout': 0.0, 'bidirectional': True}

Best Validation Accuracy: 0.9900

Best Validation F1-Score: 0.9900

RNN_2 - Best one on kaggle

Training with hidden_dim=64, num_layers=4, lr=0.001, batch_size=32, dropout=0.0, bidirectional=True

Validation Accuracy: 0.9925

Validation Accuracy: 0.9925

Training with hidden_dim=256, num_layers=3, lr=0.001, batch_size=32, dropout=0.0, bidirectional=True

Validation Accuracy: 0.9929

Validation Accuracy: 0.9929

GridSearch 3:

Best Parameters found: {'hidden_dim': 256, 'num_layers': 3, 'learning_rate': 0.001, 'batch_size': 32, 'bidirectional': True, dropout: 0.2}

Best Validation Accuracy: 0.9912

Best Validation F1-Score: 0.9912